In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
# !curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
# conda_installer.install()
conda_installer.install()
# !/root/miniconda/bin/conda info -e
import rdkit

from deepchem.feat.graph_features import atom_features as get_atom_features

all packages are already installed


In [3]:
sample_structure = rdkit.Chem.rdmolfiles.MolFromPDBFile('../Datasets/pdb-test/1a1e.pdb')

In [4]:
df = pd.read_csv('../Datasets/pdbbind.csv')

In [5]:
cols = [x for x in df.columns if (x.__contains__('gb-') or x.__contains__('vdwaals')) and not x.__contains__('etot')]

In [6]:
sample_info = df[df['complex-name'] == '1a1e'][cols].to_numpy()
sample_info = np.array(sample_info[0])

In [7]:
sample_info

array([ 7.03070120e+03, -1.35150036e+04, -3.34937910e+03,  5.65130000e+01,
        7.18952230e+03, -1.33669577e+04, -3.37817030e+03,  5.62103000e+01,
       -1.58821000e+02, -9.66715000e+01, -6.91305000e+01,  4.45980000e+00,
       -1.95158170e+03, -1.90104860e+03, -1.14008000e+01])

In [8]:
def convertBONDtoOneHotVector(bond_type):
    BOND_DICT = {dir(rdkit.Chem.rdchem.BondType)[i]: i for i in range(22)}
    one_hot = np.zeros(22)
    one_hot[BOND_DICT[str(bond_type)]] = 1
    return one_hot

In [9]:
atom_features = []
adjacency_list = []
for atom in sample_structure.GetAtoms():
    new_feature = get_atom_features(atom).tolist()
    position = sample_structure.GetConformer().GetAtomPosition(atom.GetIdx())
    new_feature += [atom.GetMass(), atom.GetAtomicNum()]
    new_feature += [position.x, position.y, position.z]
    atom_features.append(new_feature)
atom_features = np.array(atom_features)

In [10]:
for atom in sample_structure.GetAtoms():
    atom_bonds = []
    for neighbor in atom.GetNeighbors():
        bond_type = sample_structure.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx()).GetBondType()
        neighbor_idx = neighbor.GetIdx()
        atom_bonds.append([neighbor_idx, convertBONDtoOneHotVector(bond_type)])
    adjacency_list.append(atom_bonds)

In [11]:
%cd ..

/home/ali/PycharmProjects/GBNN


In [12]:
from PGGCN.layers import RuleGraphConvLayer, ConvLayer

In [13]:
class ConvLayer(tf.keras.Model):
    def __init__(self, out_channel, num_features=40):
        super(ConvLayer, self).__init__()
        self.out_channel = out_channel
        self.num_features = num_features
        self.w = tf.Variable(tf.initializers.glorot_uniform()
                             (shape=[num_features, out_channel]), shape=[num_features, out_channel], trainable=True)

    def _call_single(self, inp):
        features = inp[0]
        out = [0] * self.out_channel
        for feature in features:
            feature = tf.reshape(feature, [1, -1])
            out += tf.nn.softmax(tf.matmul(feature, self.w))
        return tf.reshape(out, [-1])

    def call(self, inputs):
        output = []
        for inp in inputs:
            output.append(self._call_single(inp))
        return tf.reshape(output, [len(inputs), -1])

In [14]:
class PGGCN(tf.keras.Model):
    def __init__(self, num_atom_features = 80, r_out_channel = 40, c_out_channel = 1024):
        super().__init__()
        self.ruleGraphConvLayer = RuleGraphConvLayer(r_out_channel, num_atom_features)
        self.conv = ConvLayer(c_out_channel)
        self.dense1 = tf.keras.layers.Dense(650, activation='sigmoid')
        self.dense2 = tf.keras.layers.Dense(1, activation='relu')
        self.dense3 = tf.keras.layers.Dense(1,
                 kernel_initializer=tf.keras.initializers.Constant([.5, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1]),
                 bias_initializer=tf.keras.initializers.Zeros(), activation=tf.keras.activations.relu)

    def set_physics_info(self, info):
        self.physics_info = info.reshape([-1, 15])

    def addRule(self, rule, start_index, end_index = None):
        self.ruleGraphConvLayer.addRule(rule, start_index, end_index)

    def call(self, inputs):
        x = self.ruleGraphConvLayer(inputs)
        x = self.conv(x)
        x = self.dense1(x)
        model_var = self.dense2(x)
        print("model var shape ", model_var.shape)
        merged = tf.concat([model_var, self.physics_info], axis=1)
        print("merged shape", merged.shape)
        print("model var", model_var)
        out = self.dense3(merged)
        return out

m = PGGCN()
m.addRule("sum", 0, 75)
m.addRule("multiply", 75, 77)
m.addRule("distance", 77, 80)
m.set_physics_info(sample_info)

2022-12-20 09:10:19.689283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-20 09:10:19.689318: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-20 09:10:19.689341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ali-ThinkPad-E14-Gen-2): /proc/driver/nvidia/version does not exist
2022-12-20 09:10:19.689678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
out = m([[atom_features, adjacency_list]])

model var shape  (1, 1)
merged shape (1, 16)
model var tf.Tensor([[0.3215393]], shape=(1, 1), dtype=float32)


In [34]:
df[df['complex-name'] == '1a1e']

,complex-name,gb-complex-etot,gb-complex-1-4-eel,gb-complex-eelec,gb-complex-egb,gb-complex-esurf,gb-protein-etot,gb-protein-1-4-eel,gb-protein-eelect,gb-protein-egb,...,pb-protein-epb,pb-protein-ecavity,pb-ligand-etot,pb-ligand-vdwaals,pb-ligand-eelec,pb-ligand-epb,pb-ligand-ecavity,ddg,enthalpy-gb,entropy
1,1a1e,-9777.1684,7030.7012,-13515.0036,-3349.3791,56.513,-9499.3955,7189.5223,-13366.9577,-3378.1703,...,-3333.0961,56.2103,-173.519,-11.4008,-96.6715,-69.9064,4.4598,-8.289306,3.2580999999991,11.5474063349991


In [16]:
out

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[3.4177675]], dtype=float32)>